## Imports

In [2]:
# Imports
import requests
import re
from bs4 import BeautifulSoup

## Misc. methods

In [8]:
# Filters the content out of a minutes-page (FOMC)
def filter_text(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    text = soup.find(id = "article").get_text()
    return text

## Retrieving current FOMC minutes (2013 - today)

In [46]:
# Get page content
current_archive_page = requests.get("https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm")
current_fomc_soup = BeautifulSoup(current_archive_page.content, 'html.parser')
current_fomc_blocks = current_fomc_soup.select(".fomc-meeting__minutes")

In [47]:
# Get URLs for minutes
# Structure of urls-list: (date, url)
current_fomc_urls = []
for m in current_fomc_blocks:
    for a in m.select("a"):
        if(".htm" in a["href"]):
            url = "https://www.federalreserve.gov" + a["href"]
            date = re.findall('\d+', url)
            current_fomc_urls.append((date, url))

In [48]:
# Extract the content of minutes pages
# Structure of current_minutes_fomc: (date, url, text)
current_minutes_fomc = []
for url in current_fomc_urls:
    article = [];
    article.append(url[0])
    article.append(url[1])
    article.append(filter_text(url[1]))
    current_minutes_fomc.append(article)

## Retrieving historical FOMC minutes (2012 and older)

In [20]:
# Get page content
historical_archive_page = requests.get("https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm")
historical_fomc_soup = BeautifulSoup(historical_archive_page.content, 'html.parser')
historical_fomc_minutes_pages = historical_fomc_soup.select(".fomc-meeting__minutes")

In [75]:
# Get url for each historical year
blocks = historical_fomc_soup.select(".panel-default")
urls_per_year = []
for block in blocks:
    for a in block.select("a"):
        urls_per_year.append("https://www.federalreserve.gov" + a["href"])

In [83]:
# Get minutes for each historical year
historical_fomc_minutes = []
for link in urls_per_year:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    blocks = soup.select(".panel-default")
    for block in blocks:
        for a in block.select("a"):
            if ("minutes" in a["href"]) and (".htm" in a["href"]) and ("#" not in a["href"]):
                if("https://www.federalreserve.gov" in a["href"]):
                    url = a["href"]
                else:
                    url= "https://www.federalreserve.gov" + a["href"]
                date = re.findall('\d+', url)
                historical_fomc_minutes.append((date, url))

# Testing Area

In [86]:
sorted(historical_fomc_minutes)

[(['19960130'], 'https://www.federalreserve.gov/fomc/minutes/19960130.htm'),
 (['19960326'], 'https://www.federalreserve.gov/fomc/minutes/19960326.htm'),
 (['19960521'], 'https://www.federalreserve.gov/fomc/minutes/19960521.htm'),
 (['19960702'], 'https://www.federalreserve.gov/fomc/minutes/19960702.htm'),
 (['19960820'], 'https://www.federalreserve.gov/fomc/minutes/19960820.htm'),
 (['19960924'], 'https://www.federalreserve.gov/fomc/minutes/19960924.htm'),
 (['19961113'], 'https://www.federalreserve.gov/fomc/minutes/19961113.htm'),
 (['19961217'], 'https://www.federalreserve.gov/fomc/minutes/19961217.htm'),
 (['19970204'], 'https://www.federalreserve.gov/fomc/minutes/19970204.htm'),
 (['19970325'], 'https://www.federalreserve.gov/fomc/minutes/19970325.htm'),
 (['19970520'], 'https://www.federalreserve.gov/fomc/minutes/19970520.htm'),
 (['19970701'], 'https://www.federalreserve.gov/fomc/minutes/19970701.htm'),
 (['19970819'], 'https://www.federalreserve.gov/fomc/minutes/19970819.htm'),